In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('../data/empirical_data/prior_research.csv')
data

,study,influence,group_id,theta,pre_influence,post_influence,subject_id,task_id,original_condition,group_task
0,lorenz2011,0,081111_1557,184,430.0,422.0,1_081111_1557,lorenz2017_task_184,Solo,081111_1557_lorenz2017_task_184
1,lorenz2011,0,081111_1557,184,520.0,22.0,2_081111_1557,lorenz2017_task_184,Solo,081111_1557_lorenz2017_task_184
2,lorenz2011,0,081111_1557,184,50.0,4.0,3_081111_1557,lorenz2017_task_184,Solo,081111_1557_lorenz2017_task_184
3,lorenz2011,0,081111_1557,184,15.0,10.0,4_081111_1557,lorenz2017_task_184,Solo,081111_1557_lorenz2017_task_184
4,lorenz2011,0,081111_1557,184,1750.0,1650.0,5_081111_1557,lorenz2017_task_184,Solo,081111_1557_lorenz2017_task_184
...,...,...,...,...,...,...,...,...,...,...
20025,becker2019,1,C_3_Social_2_Repub_becker2019,4486,2000.0,2000.0,1085_C_3_Social_2_Repub_becker2019,df_becker2019_Soldiers,Social,C_3_Social_2_Repub_becker2019_df_becker2019_So...
20026,becker2019,1,C_3_Social_2_Repub_becker2019,4486,250000.0,14000.0,1100_C_3_Social_2_Repub_becker2019,df_becker2019_Soldiers,Social,C_3_Social_2_Repub_becker2019_df_becker2019_So...
20027,becker2019,1,C_3_Social_2_Repub_becker2019,-46,70.0,NaN,1100_C_3_Social_2_Repub_becker2019,df_becker2019_Unemployment,Social,C_3_Social_2_Repub_becker2019_df_becker2019_Un...
20028,becker2019,1,C_3_Social_2_Repub_becker2019,15,20.0,16.0,1111_C_3_Social_2_Repub_becker2019,df_becker2019_Military,Social,C_3_Social_2_Repub_becker2019_df_becker2019_Mi...


# getting the task big Omega

In [3]:
df_tasks = pd.read_csv('../data/empirical_data/prior_tasks_with_omega.csv')
tasks = df_tasks.set_index('task_id')
tasks.sort_values('theta')
tasks = tasks.to_dict(orient='index')
tasks

{'lorenz2017_task_184': {'study': 'lorenz2011',
  'theta': 430.0,
  'Omega_hat': 0.710699999999938},
 'lorenz2017_task_734': {'study': 'lorenz2011',
  'theta': 350.0,
  'Omega_hat': 0.8523999999999224},
 'lorenz2017_task_10067': {'study': 'lorenz2011',
  'theta': 12000.0,
  'Omega_hat': 0.7351999999999353},
 'lorenz2017_task_198': {'study': 'lorenz2011',
  'theta': 92.0,
  'Omega_hat': 0.8079999999999273},
 'lorenz2017_task_639': {'study': 'lorenz2011',
  'theta': 98.0,
  'Omega_hat': 0.8089999999999272},
 'lorenz2017_task_9272': {'study': 'lorenz2011',
  'theta': 806.0,
  'Omega_hat': 0.8813999999999192},
 'gurcay2015_task_1_3718691.0': {'study': 'gurcay2015',
  'theta': 1000000.0,
  'Omega_hat': 0.901699999999917},
 'gurcay2015_task_2_39.52': {'study': 'gurcay2015',
  'theta': 15.0,
  'Omega_hat': 0.8326999999999246},
 'gurcay2015_task_3_2723.0': {'study': 'gurcay2015',
  'theta': 1050.0,
  'Omega_hat': 0.8706999999999204},
 'gurcay2015_task_4_7825200.0': {'study': 'gurcay2015',
  't

In [4]:
#Computing the group stuff
group_data = []
for group in data.group_task.unique():
    game_data = {}
    temp_data = data[(data.group_task==group)]
    
    if not temp_data.empty:
        task_id = np.array(temp_data.task_id)[0]
        influence = np.array(temp_data.influence)[0]
        n = temp_data.shape[0]
        truth = tasks[task_id]['theta']

        revised_woc = temp_data.post_influence.mean()
        initial_woc = temp_data.pre_influence.mean()
        

        game_data = {'group_task':group,
                     'group_id': np.array(temp_data.group_id)[0],
                     'group_size':n,
                     'Omega_hat': tasks[task_id]['Omega_hat'],
                     
                     'Omega_hat': tasks[task_id]['Omega_hat'],
                     

                     'task_id': task_id,
                     'revised_woc':revised_woc,
                     'initial_woc':initial_woc,
                     'influence':influence,
                     'dataset': np.array(temp_data.study)[0],
                     'theta': truth,
                     
                     'revised_mse': (truth - revised_woc)**2,
                     'initial_mse': (truth - initial_woc)**2,

                     'revised_abs_error': np.abs(truth - revised_woc),
                     'initial_abs_error': np.abs(truth - initial_woc),
                                     }

        group_data.append(game_data)    
df_group_data = pd.DataFrame(group_data)
zscore = lambda x: (x - x.mean()) / x.std()

df_group_data['revised_mse_zscore'] = df_group_data.groupby('task_id')['revised_mse'].transform(zscore)
df_group_data['initial_mse_zscore'] = df_group_data.groupby('task_id')['initial_mse'].transform(zscore)

df_group_data['revised_abs_error_zscore'] = df_group_data.groupby('task_id')['revised_abs_error'].transform(zscore)
df_group_data['initial_abs_error_zscore'] = df_group_data.groupby('task_id')['initial_abs_error'].transform(zscore)


df_group_data['improved'] = df_group_data['initial_abs_error'] > df_group_data['revised_abs_error']
#df_group_data

In [5]:
df_group_data.to_csv('../data/empirical_data/prior_research_group.csv',index=False)

# Robustness checks

In [6]:
df_tasks = pd.read_csv('../data/empirical_data/prior_tasks_with_omega_robsutness.csv')


for omega in df_tasks.omega_as_frac.unique():
    for n_agents in df_tasks.n_agents.unique():
        tasks = df_tasks[(df_tasks.omega_as_frac==omega) & (df_tasks.n_agents==n_agents)].set_index('task_id')
        tasks.sort_values('theta')
        tasks = tasks.to_dict(orient='index')
        
        #Computing the group stuff
        group_data = []
        for group in data.group_task.unique():
            game_data = {}
            temp_data = data[(data.group_task==group)]

            if not temp_data.empty:
                task_id = np.array(temp_data.task_id)[0]
                influence = np.array(temp_data.influence)[0]
                n = temp_data.shape[0]
                truth = tasks[task_id]['theta']

                revised_woc = temp_data.post_influence.mean()
                initial_woc = temp_data.pre_influence.mean()


                game_data = {'group_task':group,
                             'group_id': np.array(temp_data.group_id)[0],
                             'group_size':n,
                             'Omega_hat': tasks[task_id]['Omega_hat'],

                             'Omega_hat': tasks[task_id]['Omega_hat'],


                             'task_id': task_id,
                             'revised_woc':revised_woc,
                             'initial_woc':initial_woc,
                             'influence':influence,
                             'dataset': np.array(temp_data.study)[0],
                             'theta': truth,

                             'revised_mse': (truth - revised_woc)**2,
                             'initial_mse': (truth - initial_woc)**2,

                             'revised_abs_error': np.abs(truth - revised_woc),
                             'initial_abs_error': np.abs(truth - initial_woc),
                                             }

                group_data.append(game_data)    
        df_group_data = pd.DataFrame(group_data)
        zscore = lambda x: (x - x.mean()) / x.std()

        df_group_data['revised_mse_zscore'] = df_group_data.groupby('task_id')['revised_mse'].transform(zscore)
        df_group_data['initial_mse_zscore'] = df_group_data.groupby('task_id')['initial_mse'].transform(zscore)

        df_group_data['revised_abs_error_zscore'] = df_group_data.groupby('task_id')['revised_abs_error'].transform(zscore)
        df_group_data['initial_abs_error_zscore'] = df_group_data.groupby('task_id')['initial_abs_error'].transform(zscore)


        df_group_data['improved'] = df_group_data['initial_abs_error'] > df_group_data['revised_abs_error']
        df_group_data.to_csv('../data/robustness_data/prior_research_group_robust_'+'omega_'+str(omega)+'_n_'+str(n_agents)+'.csv',index=False)
#df_group_data

In [7]:
df_tasks.n_agents.unique()

array([ 5, 20, 35, 50])